In [1]:
import yaml
import csv
from collections import defaultdict

In [ ]:
file_path_yaml = r'd:\CodeRun\table_statistics\data\sample.yaml'
file_path_csv = r'd:\CodeRun\table_statistics\data\logs.csv'

In [4]:
def load_yaml(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

In [7]:
def load_logs(file_path):
    logs = []
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            logs.append(row['node_id'])
    return logs

In [ ]:
def find_statistics_nodes(folders):
    statistics_nodes = set()
    def traverse(node):
        if 'link' in node:
            return
        if node.get('name') == 'statistics':
            statistics_nodes.add(node)
            return
        for key, child in node.get('values', {}).items():
            traverse(child)
    for key, node in folders.items():
        traverse(node)
    return statistics_nodes